<font size="5"> The Prediction of heart disease based on symptoms and patient characteristics </font>

<font size= "3"> DSCI 100 section 010 group 22:  Emily Chen, Mehakpreet Dhaliwal, Haoxi Jiang, Manika Mohinani.  </font>

<font size= "4"> Introduction </font>

Heart disease, a term used to refer to several types of heart conditions, is one of the leading causes of death worldwide. Factors including lifestyle, genetic predisposition, and age, can influence an individual’s likelihood of developing heart disease. By understanding the interplay between these contributors, global healthcare agencies can work towards the improvement of early detection and prevention strategies. According to health authorities and research, angina, commonly known as chest pain, is a significant factor in the diagnosis of heart disease, along with age, cholesterol levels, and blood pressure (“Heart Disease”, 2020). We aim to address the question “Can we create a model to predict whether an individual in Cleveland is likely to have heart disease based on specific symptoms and biology of patients?” We will utilise the processed Cleveland data from the dataset ‘Heart Disease’ provided by ‘UC Irvine Machine Learning Repository’, focusing on variables that are reportedly significant in diagnosis of heart disease. 

<font size= "3"> Research question: </font>

<font size= "4"> Method and result </font>

<font size= "4"> Discussion </font>

<font size= "4"> Reference </font>